The goal of the extract block will be to extract the data from the API in AWS Lambda and save the raw data, AWS event bridge will trigger the Lambda function daily.
- save in the origonal format
- compress data (will do this at the last step 
- )
- create a new partition for a new save (YYYY/MM/DD)


In [41]:
import datetime

In [42]:
current_date = datetime.datetime.now()
year = current_date.strftime('%Y')
month = current_date.strftime('%m')
day = current_date.strftime('%d')

In [43]:
print(f"{day}")


09


For the transformation: 
Need to calculate return
calculate sma200? ema 200?
calculate EPS?
sharpe index
look at 'yesterdays' transformed data and add 'todays' data
zip and parquet

In [1]:
import pandas as pd
import pyarrow as pa

In [2]:
raw_data = pd.read_json('../etl/data/raw/alpha_vantage_daily_ARGX.json')

In [ ]:
# Tranformations from the past ETL process

# Pivot so that rows are the columns
raw_data = raw_data.transpose()

# Add date as a column and reset index
raw_data.reset_index(inplace=True)

# Order so that past date is first, new dates will be appended
raw_data.sort_values('index', ascending=True, inplace= True, ignore_index=True)

# Rename columns so that they are lower case 
raw_data.columns = ['date', 'open', 'high', 'low', 'close', 'volume']


# Daily return calculation
raw_data['daily_return'] = raw_data['close'].pct_change() # add .dropna() if you want to remove the first row with NaN value
raw_data['daily_return_p'] = raw_data['daily_return'] * 100  # multiply by 100 to get percentage for the histogram distribution

# wealth index calculation
raw_data['wealth_index'] = (raw_data['daily_return']+1).cumprod() 

# rearrange columns
raw_data = raw_data[[ 'date', 'open', 'high', 'low', 'close', 'volume','daily_return', 'daily_return_p', 'wealth_index']].dropna()


raw_data


,date,open,high,low,close,volume,daily_return,daily_return_p,wealth_index
1,2025-02-13,645.78,652.23,644.54,651.34,186353.0,0.003466,0.346639,1.003466
2,2025-02-14,649.42,649.98,632.15,639.41,266816.0,-0.018316,-1.831609,0.985087
3,2025-02-18,639.41,645.00,634.73,639.00,264784.0,-0.000641,-0.064122,0.984455
4,2025-02-19,637.90,645.02,637.60,640.14,257868.0,0.001784,0.178404,0.986211
5,2025-02-20,638.81,649.97,637.14,647.49,207756.0,0.011482,1.148186,0.997535
...,...,...,...,...,...,...,...,...,...
95,2025-07-01,555.38,563.36,541.95,542.47,606606.0,-0.015874,-1.587388,0.835739
96,2025-07-02,548.20,554.40,541.99,545.11,517681.0,0.004867,0.486663,0.839806
97,2025-07-03,545.51,545.61,538.89,543.48,289592.0,-0.002990,-0.299022,0.837295
98,2025-07-07,543.10,546.73,539.78,546.04,420895.0,0.004710,0.471038,0.841239


In [4]:
# Enrichment of the data

# Daily return calculation
raw_data['daily_return'] = raw_data['close'].pct_change() # add .dropna() if you want to remove the first row with NaN value


# multiply by 100 to get percentage for the histogram distribution


In [ ]:
# # Daily variance calculation, this is one value tho ugh so may be better to just do it in streamlit - rather than saving the same value for every row
# raw_data['daily_variance'] = raw_data['daily_return'].var()
# raw_data

,date,open,high,low,close,volume,daily_id,daily_return,daily_variance
0,2025-02-12,636.02,649.96,634.79,649.09,137079.0,1,NaN,0.000556
1,2025-02-13,645.78,652.23,644.54,651.34,186353.0,2,0.003466,0.000556
2,2025-02-14,649.42,649.98,632.15,639.41,266816.0,3,-0.018316,0.000556
3,2025-02-18,639.41,645.00,634.73,639.00,264784.0,4,-0.000641,0.000556
4,2025-02-19,637.90,645.02,637.60,640.14,257868.0,5,0.001784,0.000556
...,...,...,...,...,...,...,...,...,...
95,2025-07-01,555.38,563.36,541.95,542.47,606606.0,96,-0.015874,0.000556
96,2025-07-02,548.20,554.40,541.99,545.11,517681.0,97,0.004867,0.000556
97,2025-07-03,545.51,545.61,538.89,543.48,289592.0,98,-0.002990,0.000556
98,2025-07-07,543.10,546.73,539.78,546.04,420895.0,99,0.004710,0.000556


In [10]:
raw_data['wealth_index'] = (raw_data['daily_return']+1).cumprod() 
raw_data


,date,open,high,low,close,volume,daily_id,daily_return,daily_variance,wealth_index
0,2025-02-12,636.02,649.96,634.79,649.09,137079.0,1,NaN,0.000556,NaN
1,2025-02-13,645.78,652.23,644.54,651.34,186353.0,2,0.003466,0.000556,1.003466
2,2025-02-14,649.42,649.98,632.15,639.41,266816.0,3,-0.018316,0.000556,0.985087
3,2025-02-18,639.41,645.00,634.73,639.00,264784.0,4,-0.000641,0.000556,0.984455
4,2025-02-19,637.90,645.02,637.60,640.14,257868.0,5,0.001784,0.000556,0.986211
...,...,...,...,...,...,...,...,...,...,...
95,2025-07-01,555.38,563.36,541.95,542.47,606606.0,96,-0.015874,0.000556,0.835739
96,2025-07-02,548.20,554.40,541.99,545.11,517681.0,97,0.004867,0.000556,0.839806
97,2025-07-03,545.51,545.61,538.89,543.48,289592.0,98,-0.002990,0.000556,0.837295
98,2025-07-07,543.10,546.73,539.78,546.04,420895.0,99,0.004710,0.000556,0.841239


In [20]:
raw_data.to_parquet(f'../etl/data/processed/alpha_vantage_daily_ARGX.parquet', index=False)